In [1]:
# Dependencies
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv("nutrition.csv")
# df = pd.read_csv("nutrition.csv", index_col ="NDB_No")

In [3]:
df.head()

,FdGrp_Cd,Long_Desc,NDB_No,Shrt_Desc,Water_g,Energ_Kcal,Protein_g,Lipid_Tot_g,Ash_g,Carbohydrt_g,...,Vit_K_mcg,FA_Sat_g,FA_Mono_g,FA_Poly_g,Cholestrl_g,GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,100,"Butter, salted",1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,...,7.0,51.368,21.021,3.043,215,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0
1,100,"Butter, whipped, with salt",1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,1.62,2.87,...,4.6,45.390,19.874,3.331,225,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0
2,100,"Butter oil, anhydrous",1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,...,8.6,61.924,28.732,3.694,256,12.80,1 tbsp,205.0,1 cup,0
3,100,"Cheese, blue",1004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,...,2.4,18.669,7.778,0.800,75,28.35,1 oz,17.0,1 cubic inch,0
4,100,"Cheese, brick",1005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,...,2.5,18.764,8.598,0.784,94,132.00,"1 cup, diced",113.0,"1 cup, shredded",0


In [4]:
# df.loc[1001]
# df.loc[1129]

In [5]:
df.columns

Index(['FdGrp_Cd', 'Long_Desc', 'NDB_No', 'Shrt_Desc', 'Water_g', 'Energ_Kcal',
       'Protein_g', 'Lipid_Tot_g', 'Ash_g', 'Carbohydrt_g', 'Fiber_TD_g',
       'Sugar_Tot_g', 'Calcium_mg', 'Iron_mg', 'Magnesium_mg', 'Phosphorus_mg',
       'Potassium_mg', 'Sodium_g', 'Zinc_mg', 'Copper_mg', 'Manganese_mg',
       'Selenium_mcg', 'Vit_C_mg', 'Thiamin_mg', 'Riboflavin_mg', 'Niacin_mg',
       'Panto_Acid_mg', 'Vit_B6_mg', 'Folate_Tot_mcg', 'Folic_Acid_mcg',
       'Food_Folate_mcg', 'Folate_DFE_mcg', 'Choline_Tot_mg', 'Vit_B12_mcg',
       'Vit_A_IU', 'Vit_A_RAE', 'Retinol_mcg', 'Alpha_Carot_mcg',
       'Beta_Carot_mcg', 'Beta_Crypt_mcg', 'Lycopene_mcg', 'Lut+Zea_mcg',
       'Vit_E_mg', 'Vit_D_mcg', 'Vit_D_IU', 'Vit_K_mcg', 'FA_Sat_g',
       'FA_Mono_g', 'FA_Poly_g', 'Cholestrl_g', 'GmWt_1', 'GmWt_Desc1',
       'GmWt_2', 'GmWt_Desc2', 'Refuse_Pct'],
      dtype='object')

In [6]:
# food_data = data.set_index('Shrt_Desc').T.to_dict()
#food_data.keys()
#X = [v for k,v in df.items()]
# food_data
#food_data.keys()
# item_list = [i for i in food_data.keys()]
# item_list[0]

# Change the columns to per calorie value

In [7]:
df["Protein/cal"] = df["Protein_g"] / df["Energ_Kcal"]
df["Carbohydrtes/cal"] = df["Carbohydrt_g"] / df["Energ_Kcal"]
df["Sodium/cal"] = df["Sodium_g"]/ df["Energ_Kcal"]
df["Cholestrol/cal"] = df["Cholestrl_g"]/ df["Energ_Kcal"]
df["Sugar/cal"] = df["Sugar_Tot_g"]/ df["Energ_Kcal"]
# df["Fiber/cal"] = df["Fiber_TD_g"]/ df["Energ_Kcal"]

In [8]:
df_percalorie = df[["Shrt_Desc", "Protein/cal", "Protein_g", "Carbohydrtes/cal", "Sodium/cal", "Cholestrol/cal", "Sugar/cal"]]

In [9]:
#df_percalorie = df[["Shrt_Desc", 'Protein_g', "Protein/cal"]]

In [10]:
df_percalorie.head()

,Shrt_Desc,Protein/cal,Protein_g,Carbohydrtes/cal,Sodium/cal,Cholestrol/cal,Sugar/cal
0,"BUTTER,WITH SALT",0.001185,0.85,0.000084,0.896792,0.299861,0.000084
1,"BUTTER,WHIPPED,W/ SALT",0.000682,0.49,0.003997,0.811978,0.313370,0.000084
2,"BUTTER OIL,ANHYDROUS",0.000320,0.28,0.000000,0.002283,0.292237,0.000000
3,"CHEESE,BLUE",0.060623,21.40,0.006629,3.246459,0.212465,0.001416
4,"CHEESE,BRICK",0.062642,23.24,0.007520,1.509434,0.253369,0.001375


## Removing null values from dataframe

In [11]:
df_percalorie.isnull().sum()

Shrt_Desc            0
Protein/cal         36
Protein_g            0
Carbohydrtes/cal    31
Sodium/cal           9
Cholestrol/cal      39
Sugar/cal           38
dtype: int64

In [12]:
df_percalorie = df_percalorie.dropna(how='any',axis=0)

In [13]:
df_percalorie.isnull().sum()

Shrt_Desc           0
Protein/cal         0
Protein_g           0
Carbohydrtes/cal    0
Sodium/cal          0
Cholestrol/cal      0
Sugar/cal           0
dtype: int64

In [14]:
df_percalorie.shape

(8751, 7)

## Cosine similarity

In [15]:
# X = df_percalorie[['Protein/cal', 'Carbohydrtes/cal', 'Sodium/cal', 'Cholestrol/cal', 'Sugar/cal']]

In [16]:
X = df_percalorie[['Protein/cal']].values

In [17]:
X_norm = Normalizer().fit_transform(X)

In [18]:
# eggs
# df_percalorie.loc[1129]

In [19]:
print(int(df_percalorie[df_percalorie['Shrt_Desc']=='SOY PROT ISOLATE,K TYPE'].index.values))

5009


In [20]:
# X_norm[5009][0]

1.0

In [21]:
# food with max protein values
df_sort = df_percalorie.sort_values("Protein/cal", ascending = False)
df_sort.head(10)

,Shrt_Desc,Protein/cal,Protein_g,Carbohydrtes/cal,Sodium/cal,Cholestrol/cal,Sugar/cal
5009,"SOY PROT ISOLATE,K TYPE",0.275140,88.32,0.008069,0.155763,0.000000,0.0
4858,SOY PROTEIN ISOLATE,0.263642,88.32,0.000000,3.000000,0.000000,0.0
6204,"GELATINS,DRY PDR,UNSWTND",0.255522,85.60,0.000000,0.585075,0.000000,0.0
4473,"BEVERAGES,ENERGY DRK,VAULT ZERO,SUGAR-FREE,CIT...",0.250000,0.25,0.700000,14.000000,0.000000,0.0
4734,"FISH,POLLOCK,ALASKA,RAW (NOT PREVIOUSLY FROZEN)",0.245286,17.17,0.000000,2.271429,0.871429,0.0
4736,"FISH,COD,PACIFIC,RAW (NOT PREVIOUSLY FROZEN)",0.243611,17.54,0.000000,1.513889,0.638889,0.0
4737,"FISH,COD,PACIFIC,CKD (NOT PREVIOUSLY FROZEN)",0.243095,20.42,0.000000,1.595238,0.726190,0.0
4735,"FISH,POLLOCK,ALASKA,CKD (NOT PREVIOUSLY FROZEN)",0.242750,19.42,0.000000,2.075000,0.925000,0.0
8328,"SEAL,BEARDED (OOGRUK),MEAT,RAW (ALASKA NATIVE)",0.242727,26.70,0.000000,0.000000,0.909091,0.0
4739,"CRUSTACEANS,SHRIMP,CKD (NOT PREVIOUSLY FROZEN)",0.242222,23.98,0.002020,1.121212,1.909091,0.0


In [22]:
# df_percalorie.loc[16422]

In [22]:
# df_percalorie
# NDB_No

## Display list of 10 similar items

In [23]:
# df_percalorie

In [23]:
# X_norm[16422][0]
# dfb = int(df_percalorie[df_percalorie['NDB_No']==16422].index[0])
# dfb
# dfbb = int(df[df['A']==8].index[0])
print(int(df_percalorie[df_percalorie['Shrt_Desc']=='SOY PROT ISOLATE,K TYPE'].index.values))

5009


In [25]:
# df.index[df['BoolCol']].tolist()

In [31]:
df_sort.head(10)

,Shrt_Desc,Protein/cal,Protein_g,Carbohydrtes/cal,Sodium/cal,Cholestrol/cal,Sugar/cal
5009,"SOY PROT ISOLATE,K TYPE",0.275140,88.32,0.008069,0.155763,0.000000,0.0
4858,SOY PROTEIN ISOLATE,0.263642,88.32,0.000000,3.000000,0.000000,0.0
6204,"GELATINS,DRY PDR,UNSWTND",0.255522,85.60,0.000000,0.585075,0.000000,0.0
4473,"BEVERAGES,ENERGY DRK,VAULT ZERO,SUGAR-FREE,CIT...",0.250000,0.25,0.700000,14.000000,0.000000,0.0
4734,"FISH,POLLOCK,ALASKA,RAW (NOT PREVIOUSLY FROZEN)",0.245286,17.17,0.000000,2.271429,0.871429,0.0
4736,"FISH,COD,PACIFIC,RAW (NOT PREVIOUSLY FROZEN)",0.243611,17.54,0.000000,1.513889,0.638889,0.0
4737,"FISH,COD,PACIFIC,CKD (NOT PREVIOUSLY FROZEN)",0.243095,20.42,0.000000,1.595238,0.726190,0.0
4735,"FISH,POLLOCK,ALASKA,CKD (NOT PREVIOUSLY FROZEN)",0.242750,19.42,0.000000,2.075000,0.925000,0.0
8328,"SEAL,BEARDED (OOGRUK),MEAT,RAW (ALASKA NATIVE)",0.242727,26.70,0.000000,0.000000,0.909091,0.0
4739,"CRUSTACEANS,SHRIMP,CKD (NOT PREVIOUSLY FROZEN)",0.242222,23.98,0.002020,1.121212,1.909091,0.0


In [28]:
similarities = cosine_similarity(X_norm[5009].reshape(1,-1), X_norm)
k = 10
result = np.sort(np.argpartition(similarities[0], len(similarities[0]) - k)[-k:])

#idxmax = np.arange(start=simularities[0], stop = simularities[5] , step=1)
df_percalorie.iloc[result]

,Shrt_Desc,Protein/cal,Protein_g,Carbohydrtes/cal,Sodium/cal,Cholestrol/cal,Sugar/cal
2963,"EDAMAME,FRZ,UNPREP",0.102936,11.22,0.069817,0.055046,0.000000,0.022752
2968,"GINGER ROOT,RAW",0.022750,1.82,0.222125,0.162500,0.000000,0.021250
2970,"GOURD,WHITE-FLOWERED (CALABASH),CKD,BLD,DRND,W...",0.040000,0.60,0.246000,0.133333,0.000000,0.000000
2971,"GOURD,DISHCLOTH (TOWELGOURD),RAW",0.060000,1.20,0.217500,0.150000,0.000000,0.101000
2972,"GOURD,DISHCLOTH (TOWELGOURD),CKD,BLD,DRND,WO/SALT",0.011786,0.66,0.256071,0.375000,0.000000,0.092321
2973,"DRUMSTICK LEAVES,RAW",0.146875,9.40,0.129375,0.140625,0.000000,0.000000
2974,"DRUMSTICK LEAVES,CKD,BLD,DRND,WO/ SALT",0.087833,5.27,0.185833,0.150000,0.000000,0.016667
2975,"HYACINTH-BEANS,IMMAT SEEDS,RAW",0.045652,2.10,0.199783,0.043478,0.000000,0.088696
2976,"HYACINTH-BEANS,IMMAT SEEDS,CKD,BLD,DRND,WO/SALT",0.059000,2.95,0.184000,0.040000,0.000000,0.000000
8789,"TURTLE,GREEN,RAW",0.222472,19.80,0.000000,0.764045,0.561798,0.000000


In [32]:
np.sort(np.argpartition(similarities[0], len(similarities[0]) - k)[-k:])

array([2960, 2965, 2967, 2968, 2969, 2970, 2971, 2972, 2973, 8750],
      dtype=int32)

In [33]:
result

array([2960, 2965, 2967, 2968, 2969, 2970, 2971, 2972, 2973, 8750],
      dtype=int32)

In [34]:
similarities[0]

array([1., 1., 1., ..., 0., 1., 1.])

In [35]:
# df = pd.DataFrame({
#     "eggs": [1, 2, 3, 5],
#     "bacon": [12, 123, 3, 5],
#     "spam": [12, 23, 4, 5]
# })
# # index 0 eggs, 1 bacon, 2 spam
# X_norm = Normalize().fit_transform(X)
# cosine_simularity(X_norm[0], X_norm[~0])
# X_norm[~0].dot(X_norm[0].T)
# X_sim = X_norm.dot(X_norm.T)
# [
#     [.123, 1, 1],
#     [.240, 1, *],
#     [1, *, *]
# ]
# sim_eggs = X_sim[0]
# # search for the index that is closest but not the egg index